In [41]:
import pandas as pd
import numpy as np

from sklearn.datasets import  fetch_california_housing
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

In [39]:
califorina_housing = fetch_california_housing(as_frame=True)
cali_df = califorina_housing.frame

cali_df.columns = cali_df.columns.str.replace(' ','')

In [23]:
cali_df.sample(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
20316,4.1279,17.0,5.001253,1.004386,6437.0,4.033208,34.17,-119.16,2.239
2825,5.6235,12.0,6.259528,0.947368,1754.0,3.183303,35.41,-119.09,1.306
11856,3.0625,19.0,5.607143,1.078571,366.0,2.614286,39.80,-120.15,0.825


In [40]:
cali_df['MedInc'].max()

np.float64(15.0001)

# Feature Engineering
feature engineering = menambah feature baru dari yg udah ada

In [5]:
x = cali_df.drop('MedHouseVal', axis=1)
y = cali_df['MedHouseVal']

In [6]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(x, y, test_size=0.2, random_state=42)

one hot endcoding = kita akan membagi katogori, klo kita ada 4 catogory, kita pisah kan semua nya jadi katogori baru, kolomnya dalam binary classifkasi, yes or no.<br>
apakah katogori 1 ya atau tidak<br>
apakah katogori 2 ya atau tidak<br>

## Fitur Categorical Buatan dengan Binning

In [ ]:
# Kita akan membagi Median Income menjadi 4 kategori (bins)
bins = [0, 2.5, 5.0, 7.5, np.inf]
labels = ['low_income', 'medium_income', 'high_income', 'very_high_income']

X_Train['income_categorical'] = pd.cut(X_Train['MedInc'], bins=bins, labels=labels, right=False)
X_Test['income_categorical'] = pd.cut(X_Train['MedInc'], bins=bins, labels=labels, right=False)

# ini fungsi right=  0- 2.5   2.5-5.0   5.0-7.5   7.5-infinit

In [24]:
X_Train[['MedInc', 'income_categorical']].sample(3)

,MedInc,income_categorical
5626,3.2115,medium_income
15917,2.3087,low_income
15559,3.0054,medium_income


# one hot encoding

In [ ]:
# kategori tadi kita one hot encoding
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [10]:
# fit transofom untuk modelnya belajar training data. transform untuk mengubah datanya berdaarkan apa yg dipelari
# biar model belajar di xtrain bukan di xtest 

# di training data belajar nya lebih gede daripada,  transform atau test data

X_Train_cate = encoder.fit_transform(X_Train[['income_categorical']])
X_Test_cate = encoder.transform(X_Test[['income_categorical']])

In [11]:
X_Train_cate

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]], shape=(16512, 4))

In [26]:
# Buat DataFrame dari hasil encoding
feature_name = encoder.get_feature_names_out(['income_categorical'])
X_train_cat_df = pd.DataFrame(X_Train_cate, columns=feature_name, index=X_Train.index)
X_test_cat_df = pd.DataFrame(X_Test_cate, columns=feature_name, index=X_Test.index)

In [38]:
X_test_cat_df.sample(2)

,income_categorical_high_income,income_categorical_low_income,income_categorical_medium_income,income_categorical_very_high_income
7901,0.0,0.0,0.0,0.0
14340,0.0,0.0,0.0,0.0


## Polynomial Feature HouseAge and Population

In [ ]:
# Polynomial Features untuk fitur numerik
# Pilih fitur numerik yang relevan untuk dikombinasikan (misal: HouseAge dan Population)
poly = PolynomialFeatures(degree=2, include_bias=False)
# fungsi degree untuk mancari non linaerisis

In [15]:
X_Train_poly = poly.fit_transform(X_Train[['HouseAge', 'Population']])
X_Test_poly = poly.transform(X_Test[['HouseAge', 'Population']])

In [16]:
poly_feature_names =poly.get_feature_names_out(['HouseAge', 'Population'])
X_Train_poly_df = pd.DataFrame(X_Train_poly, columns=poly_feature_names, index=X_Train.index)
X_Test_poly_df = pd.DataFrame(X_Test_poly, columns=poly_feature_names, index=X_Test.index)

In [37]:
X_Train_poly_df.sample(3)

,HouseAge,Population,HouseAge^2,HouseAge Population,Population^2
18724,5.0,2965.0,25.0,14825.0,8791225.0
4384,44.0,1783.0,1936.0,78452.0,3179089.0
1494,19.0,1317.0,361.0,25023.0,1734489.0


In [ ]:
# Gabungkan semua fitur yang sudah di-engineer
# Hapus fitur yang sudah diolah dan fitur buatan
x_train_cleaned = X_Train.drop(columns=['income_categorical', 'HouseAge', 'Population'])
x_test_cleaned = X_Test.drop(columns=['income_categorical', 'HouseAge', 'Population'])

In [28]:
x_train_engineered = pd.concat([x_train_cleaned, X_train_cat_df, X_Train_poly_df], axis=1)
x_test_engineered = pd.concat([x_test_cleaned, X_test_cat_df, X_Test_poly_df], axis=1)

In [36]:
x_train_engineered.sample(3)

,MedInc,AveRooms,AveBedrms,AveOccup,Latitude,Longitude,income_categorical_high_income,income_categorical_low_income,income_categorical_medium_income,income_categorical_very_high_income,HouseAge,Population,HouseAge^2,HouseAge Population,Population^2
5659,4.1940,5.092857,1.076786,2.017857,33.73,-118.30,0.0,0.0,1.0,0.0,47.0,1130.0,2209.0,53110.0,1276900.0
10907,2.5651,3.794745,1.037767,4.821018,33.75,-117.85,0.0,0.0,1.0,0.0,27.0,2936.0,729.0,79272.0,8620096.0
8994,4.1375,5.070611,1.045802,2.185115,33.99,-118.36,0.0,0.0,1.0,0.0,43.0,1145.0,1849.0,49235.0,1311025.0


# Feature Scaling

In [19]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train_engineered)
X_test_scaled =  scaler.transform(x_test_engineered)

In [ ]:
feature_names = scaler.get_feature_names_out()

X_train_scaled_standard_df = pd.DataFrame(X_train_scaled, columns=feature_names, index=X_Train.index)
X_test_scaled_standard_df = pd.DataFrame(X_test_scaled, columns=feature_names, index=X_Test.index)

In [34]:
X_train_scaled_standard_df.describe()

,MedInc,AveRooms,AveBedrms,AveOccup,Latitude,Longitude,income_categorical_high_income,income_categorical_low_income,income_categorical_medium_income,income_categorical_very_high_income,HouseAge,Population,HouseAge^2,HouseAge Population,Population^2
count,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04
mean,-6.519333e-17,-1.981081e-16,-1.707291e-16,4.936566e-17,6.400995e-17,1.753335e-15,6.562365e-18,-3.614680e-17,9.337922e-17,-3.571648e-17,-9.251859e-18,-2.151595e-19,-1.947193e-17,-3.378004e-17,1.180688e-17
std,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00
min,-1.775438e+00,-1.904386e+00,-1.762117e+00,-2.076846e-01,-1.447697e+00,-2.377207e+00,-4.608946e-01,-5.479597e-01,-1.100201e+00,-2.202592e-01,-2.190766e+00,-1.251913e+00,-1.299180e+00,-1.442046e+00,-2.349694e-01
25%,-6.900689e-01,-4.118373e-01,-2.081645e-01,-5.770769e-02,-8.018107e-01,-1.110749e+00,-4.608946e-01,-5.479597e-01,-1.100201e+00,-2.202592e-01,-8.417859e-01,-5.606339e-01,-8.693349e-01,-6.756449e-01,-1.910121e-01
50%,-1.758995e-01,-8.350905e-02,-1.094160e-01,-2.415892e-02,-6.473597e-01,5.346501e-01,-4.608946e-01,-5.479597e-01,9.089250e-01,-2.202592e-01,3.108328e-02,-2.281865e-01,-1.813166e-01,-1.883631e-01,-1.388033e-01
75%,4.686502e-01,2.621376e-01,8.455177e-03,1.580865e-02,9.720351e-01,7.839530e-01,-4.608946e-01,-5.479597e-01,9.089250e-01,-2.202592e-01,6.658972e-01,2.634487e-01,5.213405e-01,4.537180e-01,-2.460932e-02
max,5.839268e+00,5.716655e+01,5.664727e+01,1.071164e+02,2.951816e+00,2.628794e+00,2.169694e+00,1.824952e+00,9.089250e-01,4.540106e+00,1.856173e+00,3.012743e+01,2.297945e+00,1.824128e+01,8.966953e+01


In [30]:
from sklearn.metrics import r2_score

# Model 1: Menggunakan data MENTAH
model_raw = LinearRegression()

# Kita harus mengambil data numerik saja, karena model tidak bisa memproses data kategorikal
X_train_raw = X_Train.select_dtypes(include=np.number)
X_test_raw = X_Test.select_dtypes(include=np.number)
model_raw.fit(X_train_raw, Y_Train)
y_pred_raw = model_raw.predict(X_test_raw)
print("Metrik Model dengan Data Mentah:")
print(f"MSE: {mean_squared_error(Y_Test, y_pred_raw):.2f}")
print(f"NMSE: {(mean_squared_error(Y_Test, y_pred_raw) / (y_pred_raw.max() - y_pred_raw.min())):.2f}")
print(f"R2 Score: {r2_score(Y_Test, y_pred_raw):.2f}")

Metrik Model dengan Data Mentah:
MSE: 0.56
NMSE: 0.04
R2 Score: 0.58


In [31]:
# Model 2: Menggunakan data setelah FEATURE ENGINEERING (tanpa scaling)

model_eng = LinearRegression()
model_eng.fit(x_train_engineered, Y_Train)
y_pred_fe = model_eng.predict(x_test_engineered)
print("Metrik Model dengan Data Feture Engineering:")
print(f"MSE: {mean_squared_error(Y_Test, y_pred_fe):.2f}")
print(f"NMSE: {(mean_squared_error(Y_Test, y_pred_fe) / (y_pred_fe.max() - y_pred_fe.min())):.2f}")
print(f"R2 Score: {r2_score(Y_Test, y_pred_fe):.2f}")

Metrik Model dengan Data Feture Engineering:
MSE: 0.60
NMSE: 0.05
R2 Score: 0.55


In [33]:
# Model 3: Menggunakan data setelah FEATURE ENGINEERING + FEATURE SCALING (StandardScaler)

model_scaled = LinearRegression()
model_scaled.fit(X_train_scaled, Y_Train)
y_pred_scaled = model_scaled.predict(X_test_scaled)
print("Metrik Model dengan Data setelah Feature Engineering & Scaling:")
print(f"MSE: {mean_squared_error(Y_Test, y_pred_scaled):.2f}")
print(f"NMSE: {(mean_squared_error(Y_Test, y_pred_fe) / (y_pred_scaled.max() - y_pred_scaled.min())):.2f}")
print(f"R2 Score: {r2_score(Y_Test, y_pred_scaled):.2f}")

Metrik Model dengan Data setelah Feature Engineering & Scaling:
MSE: 0.58
NMSE: 0.05
R2 Score: 0.56
